* take two different tokenizers mono and multi
* write code to run some data through the tokenizers OUTPUT IN TEXTFORM
* check how gpt4 tokenizes


*** ALL ANSWERS AT THE BOTTOM OF THE NOTEBOOK***

In [1]:
# install transformers

!pip install --quiet transformers wikipedia-api


In [2]:
# import autotokenizer and wikipedia-api
from transformers import AutoTokenizer
import wikipediaapi

In [3]:
# single language
MONO="bert-base-cased"
mono_tokenizer=AutoTokenizer.from_pretrained(MONO)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# multilanguage
MULTI="bert-base-multilingual-cased"
multi_tokenizer=AutoTokenizer.from_pretrained(MULTI)

Write code to load the selected tokenizers and tokenize text using these (output is expected to be subword tokens in text form, not numbers). Select a piece of text (e.g. Wikipedia page or news article) written in your target language and tokenize it separately using both models. Inspect whether the tokenization results differ. How many subwords did each produce?


In [5]:
# use wikipedia-api

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')

page_py = wiki_wiki.page('Python_(programming_language)')

In [6]:
summary = page_py.summary.replace("\n", "")

In [7]:
summary

'Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.'

This works as my corpus

In [8]:
from collections import defaultdict

word_freqs = defaultdict(int)

words_with_offsets = mono_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(summary)
new_words = [w for w, o in words_with_offsets]
for w in new_words:
  word_freqs[w] += 1

print(word_freqs)



defaultdict(<class 'int'>, {'Python': 9, 'is': 3, 'a': 4, 'high': 1, '-': 5, 'level': 1, ',': 8, 'general': 1, 'purpose': 1, 'programming': 5, 'language': 3, '.': 16, 'Its': 1, 'design': 1, 'philosophy': 1, 'emphasizes': 1, 'code': 1, 'readability': 1, 'with': 2, 'the': 6, 'use': 2, 'of': 3, 'significant': 1, 'indentation': 1, 'dynamically': 1, 'typed': 1, 'and': 4, 'garbage': 1, 'collected': 1, 'It': 2, 'supports': 1, 'multiple': 1, 'paradigms': 1, 'including': 1, 'structured': 1, '(': 1, 'particularly': 1, 'procedural': 1, ')': 1, 'object': 1, 'oriented': 1, 'functional': 1, 'often': 1, 'described': 1, 'as': 4, '"': 2, 'batteries': 1, 'included': 1, 'due': 1, 'to': 2, 'its': 1, 'comprehensive': 1, 'standard': 1, 'library': 1, 'Guido': 1, 'van': 1, 'Rossum': 1, 'began': 1, 'working': 1, 'on': 1, 'in': 6, 'late': 1, '1980s': 1, 'successor': 1, 'ABC': 1, 'first': 1, 'released': 4, 'it': 1, '1991': 1, '0': 4, '9': 1, '2': 3, 'was': 3, '2000': 1, '3': 1, '2008': 1, 'major': 1, 'revision':

In [9]:
alphas = []

for w in word_freqs.keys():
  for l in w:
    if l not in alphas:
      alphas.append(l)

alphas.sort()


In [10]:
splits = {word: [char for char in word] for word in word_freqs.keys()}

In [11]:
def compute_pair_freqs(splits) -> defaultdict:
  pair_freqs = defaultdict(int)
  for w, f in word_freqs.items():
    split = splits[w]
    if len(split) == 1:
      continue
    for i in range(len(split) - 1):
      pair = (split[i], split[i + 1])
      pair_freqs[pair] += f
  return pair_freqs

In [12]:
pair_freqs = compute_pair_freqs(splits)

# part of the dict
for i, k in enumerate(pair_freqs.keys()):
  print(f'{k}: {pair_freqs[k]}')
  if i >= 5:
    break

('P', 'y'): 9
('y', 't'): 9
('t', 'h'): 17
('h', 'o'): 9
('o', 'n'): 16
('i', 's'): 5


In [13]:

most = ""
freq = None

for p, f in pair_freqs.items():
  if freq is None or f > freq:
    most = p
    freq = f

print(most, freq)


('i', 'n') 19




---



In [14]:
vocab = alphas.copy()

In [15]:

merges = {most: "".join(most)}

In [16]:
vocab.append(merges[most])


In [17]:
def merge_pair(a,b, splits):
  for w in word_freqs:
    split = splits[w]
    if len(split) == 1:
      continue

    i = 0
    while i < len(split) - 1:
      if split[i] == a and split[i+1] == b:
        split = split[:i] + [a+b]+split[i+2:]
      else:
        i += 1
    splits[w] = split
  return splits


In [18]:
splits = merge_pair(most[0], most[1], splits)

In [19]:
vocab_size = 330

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [20]:
print(merges)

{('i', 'n'): 'in', ('t', 'h'): 'th', ('o', 'n'): 'on', ('a', 's'): 'as', ('a', 'n'): 'an', ('e', 'd'): 'ed', ('l', 'e'): 'le', ('a', 'r'): 'ar', ('P', 'y'): 'Py', ('Py', 'th'): 'Pyth', ('Pyth', 'on'): 'Python', ('r', 'e'): 're', ('in', 'g'): 'ing', ('r', 'a'): 'ra', ('e', 's'): 'es', ('e', 'n'): 'en', ('p', 'r'): 'pr', ('pr', 'o'): 'pro', ('m', 'm'): 'mm', ('c', 'o'): 'co', ('th', 'e'): 'the', ('i', 's'): 'is', ('pro', 'g'): 'prog', ('prog', 'ra'): 'progra', ('progra', 'mm'): 'programm', ('programm', 'ing'): 'programming', ('a', 'g'): 'ag', ('t', 'i'): 'ti', ('an', 'd'): 'and', ('o', 'r'): 'or', ('re', 'le'): 'rele', ('rele', 'as'): 'releas', ('i', 'g'): 'ig', ('o', 's'): 'os', ('l', 'an'): 'lan', ('lan', 'g'): 'lang', ('lang', 'u'): 'langu', ('langu', 'ag'): 'languag', ('m', 'p'): 'mp', ('i', 't'): 'it', ('o', 'f'): 'of', ('l', 'y'): 'ly', ('releas', 'ed'): 'released', ('2', '0'): '20', ('v', 'e'): 've', ('languag', 'e'): 'language', ('I', 't'): 'It', ('d', 'es'): 'des', ('a', 'd'): '

MULTI

In [21]:
multi_w_freqs = defaultdict(int)

multi_words_with_offsets = multi_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(summary)
nw = [w for w,o in multi_words_with_offsets]
for w in nw:
  multi_w_freqs[w] += 1

print(multi_w_freqs)

defaultdict(<class 'int'>, {'Python': 9, 'is': 3, 'a': 4, 'high': 1, '-': 5, 'level': 1, ',': 8, 'general': 1, 'purpose': 1, 'programming': 5, 'language': 3, '.': 16, 'Its': 1, 'design': 1, 'philosophy': 1, 'emphasizes': 1, 'code': 1, 'readability': 1, 'with': 2, 'the': 6, 'use': 2, 'of': 3, 'significant': 1, 'indentation': 1, 'dynamically': 1, 'typed': 1, 'and': 4, 'garbage': 1, 'collected': 1, 'It': 2, 'supports': 1, 'multiple': 1, 'paradigms': 1, 'including': 1, 'structured': 1, '(': 1, 'particularly': 1, 'procedural': 1, ')': 1, 'object': 1, 'oriented': 1, 'functional': 1, 'often': 1, 'described': 1, 'as': 4, '"': 2, 'batteries': 1, 'included': 1, 'due': 1, 'to': 2, 'its': 1, 'comprehensive': 1, 'standard': 1, 'library': 1, 'Guido': 1, 'van': 1, 'Rossum': 1, 'began': 1, 'working': 1, 'on': 1, 'in': 6, 'late': 1, '1980s': 1, 'successor': 1, 'ABC': 1, 'first': 1, 'released': 4, 'it': 1, '1991': 1, '0': 4, '9': 1, '2': 3, 'was': 3, '2000': 1, '3': 1, '2008': 1, 'major': 1, 'revision':

In [22]:
multialphas = []

for w in multi_w_freqs.keys():
  for l in w:
    if l not in multialphas:
      multialphas.append(l)

multialphas.sort()
print(multialphas)

['"', '(', ')', ',', '-', '.', '0', '1', '2', '3', '7', '8', '9', 'A', 'B', 'C', 'G', 'I', 'P', 'R', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


In [23]:
multivocab = multialphas.copy()

In [24]:
multisplits = {w: [c for c in w] for w in multi_w_freqs.keys()}

In [25]:
mpfs = compute_pair_freqs(multisplits)

for i, k in enumerate(mpfs.keys()):
  print(f"{k}: {mpfs[k]}")
  if i >= 5:
      break

('P', 'y'): 9
('y', 't'): 9
('t', 'h'): 17
('h', 'o'): 9
('o', 'n'): 16
('i', 's'): 5


In [26]:
best_pair = ""
max_freq = None

for pair, freq in mpfs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('i', 'n') 19


In [27]:
multi_merges = {best_pair: "".join(best_pair)}

In [28]:
vocab.append(multi_merges[best_pair])

In [29]:
multisplits = merge_pair(best_pair[0], best_pair[1], multisplits)

In [30]:
print(multisplits["Python"])

['P', 'y', 't', 'h', 'o', 'n']


In [31]:
vocab_size = 300

while len(multivocab) < vocab_size:
    pair_freqs = compute_pair_freqs(multisplits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    multisplits = merge_pair(*best_pair, multisplits)
    multi_merges[best_pair] = best_pair[0] + best_pair[1]
    multivocab.append(best_pair[0] + best_pair[1])

What are the vocabulary sizes in these models? Keeping in mind that the multilingual model is trained on several languages (how many?), how do these compare?

In [32]:
# single language
print("vocabulary size for mono: ", mono_tokenizer.vocab_size)

vocabulary size for mono:  28996


In [33]:
# multi language
print("count of different languages: 104") # from https://huggingface.co/google-bert/bert-base-multilingual-cased
print("vocabulary size for multilanguage: ",multi_tokenizer.vocab_size)

count of different languages: 104
vocabulary size for multilanguage:  119547


In [34]:
print(f'Token count for monolanguage subword tokenizer: {len(merges)}')
print(f'Token count for multilanguage subword tokenizer: {len(multi_merges)}')

Token count for monolanguage subword tokenizer: 286
Token count for multilanguage subword tokenizer: 257


This webpage can be used to visualize how ChatGPT tokenizes text. Try what it does with English and Finnish (or any other smaller language) text. What is your take on this?

For text:

Write code to load the selected tokenizers and tokenize text using these (output is expected to be subword tokens in text form, not numbers). Select a piece of text (e.g. Wikipedia page or news article) written in your target language and tokenize it separately using both models. Inspect whether the tokenization results differ. How many subwords did each produce?

it generated 74 tokens

And translated to finnish:

Kirjoita koodi, joka lataa valitut tokenizerit ja tokenisoi tekstiä niiden avulla (tulosteen odotetaan olevan tekstimuotoisia sanan alaisia tokeneita, ei numeroita). Valitse kohdekielelläsi kirjoitettu teksti (esim. Wikipedia-sivu tai uutisartikkeli) ja tokenisoi se erikseen molemmilla malleilla. Tarkasta, eroavatko tokenisointitulokset toisistaan. Kuinka monta alasanaa kumpikin tuotti?

It generated 140 tokens



Answer:

In the page OpenAI says that by rule of thumb one token in english is roughly 4 chars. It seems that for finnish the char count per token is less than that. both texts have roughly the same length.